In [2]:
!pip install -qU python-dotenv
!pip install -qU langchain
!pip install -qU langchain_openai
!pip install -qU pypdf
!pip install -qU langchain_community
!pip install -qU unstructured
!pip install -qU chromadb
!pip install -qU PyPDF2
!pip install -qU langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/

In [3]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [157]:
import json
import PyPDF2
from PyPDF2 import PdfReader
from langchain_openai import OpenAI

from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks import get_openai_callback

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from textwrap import dedent
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#vector database
from langchain.vectorstores.chroma import Chroma
#from langchain_chroma import Chroma
#import chromadb


In [8]:
#embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [233]:
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=4000, temperature=0, api_key=os.environ['OPENAI_API_KEY'])
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", max_tokens=10000, temperature=0, api_key=os.environ['OPENAI_API_KEY'])

In [106]:
def preparar_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:

        pdf_reader = PyPDF2.PdfReader(pdf)
        for index, text in enumerate(pdf_reader.pages):
            doc_page = {'title': pdf + " page " + str(index + 1),
                        'content': pdf_reader.pages[index].extract_text()}
            docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Contenido y Metadatos extraidos desde los documentos")
    return content, metadata

def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=512,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documentos se separan en {len(split_docs)} partes")
    return split_docs


In [234]:
def load_pdf_to_db(split_docs):
  DB_CHROMA_PATH = 'vectorstore/'
  db = Chroma.from_documents(documents=split_docs, embedding=embeddings,
                             persist_directory=DB_CHROMA_PATH)

  db.persist()
  return db

In [11]:
pdf_folder = 'CVs'

In [12]:
lista_archivos = [
        f'/content/{pdf_folder}/CV1.pdf',
        f'/content/{pdf_folder}/CV2.pdf',
        f'/content/{pdf_folder}/CV3.pdf',
        f'/content/{pdf_folder}/CV4.pdf',
        f'/content/{pdf_folder}/CV5.pdf',
        f'/content/{pdf_folder}/CV6.pdf',
        f'/content/{pdf_folder}/CV7.pdf',
        f'/content/{pdf_folder}/CV8.pdf',
        f'/content/{pdf_folder}/CV9.pdf'
]

In [44]:
def create_embedding(new_file_trunk, new_file_pdf_path, file_persistent_dir_path):
  print(f"Creando los embedding para {file_persistent_dir_path}")
  #lectura simple de la informacion de los PDF
  pdf_reader = PdfReader(new_file_pdf_path)
  text = ""
  for page in pdf_reader.pages:
      text += page.extract_text()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len)

  chunks = text_splitter.split_text(text=text)

  new_chunks = [Document(page_content=chunk) for chunk in chunks]

  db = Chroma.from_documents(
      documents=new_chunks, embedding=embeddings, persist_directory=file_persistent_dir_path)

  db.persist()

In [39]:
for file_path in lista_archivos:
  new_file_trunk = file_path[:-len(".pdf")]
  create_embedding(new_file_trunk, file_path, os.path.join("data", new_file_trunk))

Creando los embedding para /content/CVs/CV1


<ipython-input-38-c4368e271e55>:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


Creando los embedding para /content/CVs/CV2
Creando los embedding para /content/CVs/CV3
Creando los embedding para /content/CVs/CV4
Creando los embedding para /content/CVs/CV5
Creando los embedding para /content/CVs/CV6
Creando los embedding para /content/CVs/CV7
Creando los embedding para /content/CVs/CV8
Creando los embedding para /content/CVs/CV9


In [41]:
DB_final = Chroma(embedding_function=embeddings)

<ipython-input-41-f68cf0d3466b>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  DB_final = Chroma(embedding_function=embeddings)


In [42]:
lista_folder = [os.path.join("CVs", filename[:-len(".pdf")] ) for filename in lista_archivos]
lista_folder

['/content/CVs/CV1',
 '/content/CVs/CV2',
 '/content/CVs/CV3',
 '/content/CVs/CV4',
 '/content/CVs/CV5',
 '/content/CVs/CV6',
 '/content/CVs/CV7',
 '/content/CVs/CV8',
 '/content/CVs/CV9']

In [ ]:
lista_folder = [os.path.join("CVs", filename[:-len(".pdf")] ) for filename in lista_archivos]
for db_path in lista_folder:
    print(f"Cargando documentos desde {db_path}")
    # Cargar las documentos existentes en la base de datos vectoriales .
    DB_aux = Chroma(persist_directory=db_path, embedding_function=embeddings)
    DB_aux_data = DB_aux._collection.get(include=['documents','metadatas','embeddings'])
    DB_final._collection.add(
          embeddings=DB_aux_data['embeddings'],
          metadatas=DB_aux_data['metadatas'],
          documents=DB_aux_data['documents'],
          ids=DB_aux_data['ids'])


In [286]:
retriever = DB_final.as_retriever(search_type="similarity", search_kwargs={"k": len(lista_archivos)})

In [47]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x796111111810>, search_kwargs={'k': 9})

In [287]:
chain = ConversationalRetrievalChain.from_llm(llm, retriever, return_source_documents=True)

In [323]:
query = "Experience in Machine Learning"
prompt = dedent ("""You are an expert recruiter.
  Based on the context return output string with the following JSON structure, the following structure:
  [
    {
      "name": CV name
      "email": CV email
      "contact_phone": CV phone
      "years_of_experience": CV Years of experience
      "skills": list of skills in CV
      "ai_skills": CV evaluate if candidate has experience in any branchs of Artificial Intelligence (AI, ML, GenAI). Yes/No.
      "score": Evaluate between 1 and 10 the score related to requeriments in question
    }
  ]
  context = Give information about CV.
  question = """ +query+""".
  If no matches are found, return the empty list.
""")

In [324]:
str_json_data = None
with get_openai_callback() as cb:
    response = chain({'question': prompt, 'chat_history': ''})
    str_json_data = response['answer']
    #print(response)
#str_json_data

In [325]:
str_json_data = str_json_data.replace("```json", '"')
str_json_data = str_json_data.replace("```", '"')
if str_json_data[0] == '"':
  str_json_data = str_json_data[1:]
if str_json_data[-1] == '"':
  str_json_data = str_json_data[:-1]
str_json_data

'[\n  {\n    "name": "Alice Clark",\n    "email": "Email me on Indeed",\n    "contact_phone": "Not provided",\n    "years_of_experience": "20+ years",\n    "skills": [\n      "data handling",\n      "design",\n      "development",\n      "data analysis",\n      "star/snowflake schema data modelling",\n      "database designing",\n      "scalability",\n      "back-up and recovery",\n      "writing and optimizing SQL code",\n      "Stored Procedures",\n      "creating functions",\n      "views",\n      "triggers",\n      "indexes",\n      "Microsoft Azure cloud services",\n      "Document DB",\n      "SQL Azure",\n      "Stream Analytics",\n      "Event hub",\n      "Power BI",\n      "Web Job",\n      "Web App",\n      "Azure data lake analytics (U-SQL)",\n      "big data processing",\n      "Azure data factory"\n    ],\n    "ai_skills": "Yes",\n    "score": 8\n  }\n]'

In [326]:
json_data = json.loads(str_json_data)
for data in json_data:
    print(json.dumps(data, indent=2))


{
  "name": "Alice Clark",
  "email": "Email me on Indeed",
  "contact_phone": "Not provided",
  "years_of_experience": "20+ years",
  "skills": [
    "data handling",
    "design",
    "development",
    "data analysis",
    "star/snowflake schema data modelling",
    "database designing",
    "scalability",
    "back-up and recovery",
    "writing and optimizing SQL code",
    "Stored Procedures",
    "creating functions",
    "views",
    "triggers",
    "indexes",
    "Microsoft Azure cloud services",
    "Document DB",
    "SQL Azure",
    "Stream Analytics",
    "Event hub",
    "Power BI",
    "Web Job",
    "Web App",
    "Azure data lake analytics (U-SQL)",
    "big data processing",
    "Azure data factory"
  ],
  "ai_skills": "Yes",
  "score": 8
}
